In [1]:
from langchain import PromptTemplate, LLMChain
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceBgeEmbeddings
from io import BytesIO
from langchain.document_loaders import PyPDFLoader
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore

model_name = "BAAI/bge-large-en"
# model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    # model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)


In [2]:

loader = PyPDFLoader("./backend/app/upload_files/572_qd_dhcntt29_09_2021_cvht_k2021.pdf")
documents = loader.load()

parent_splitter = RecursiveCharacterTextSplitter(chunk_size=450,chunk_overlap=200)
child_splitter = RecursiveCharacterTextSplitter(chunk_size=250,chunk_overlap=10)

vectorstore = Chroma(collection_name="split_parents", embedding_function=embeddings,collection_metadata={"hnsw:space": "cosine"})

store = InMemoryStore()
print("Vector Store Created.......")

Vector Store Created.......


In [3]:
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

In [4]:
retriever.add_documents(documents)

In [7]:
retriever.get_relevant_documents("what is title of the document?")

NameError: name 'retriever' is not defined

In [2]:

loader = PyPDFLoader("./backend/app/upload_files/Identification_of_Gender_from_Facial_Features.pdf")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=450, chunk_overlap=30)
texts = text_splitter.split_documents(documents)

vector_store2 = Chroma.from_documents(texts, embeddings, collection_metadata={"hnsw:space": "cosine"})

In [3]:
chroma_retriever = vector_store2.as_retriever(search_kwargs={"k": 2})

In [4]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever

text_splitter = RecursiveCharacterTextSplitter(chunk_size=450, chunk_overlap=200)
docs = text_splitter.split_documents(documents)

bm25_retriever = BM25Retriever.from_documents(docs)
bm25_retriever.k = 2


retriever_ensemble = EnsembleRetriever(retrievers=[bm25_retriever, chroma_retriever], weights=[0.25, 0.75])

In [9]:
docs = retriever_ensemble.get_relevant_documents("what is main content?")

In [11]:
docs

[Document(metadata={'page': 0, 'source': './backend/app/upload_files/Identification_of_Gender_from_Facial_Features.pdf'}, page_content='Copyright: ©  the author(s), publisher and licensee Technoscience Academy. This is an open -access article distributed under the \nterms of the Creative Commons Attribution Non -Commercial License, which permits unrestricted non -commercial use, \ndistribution, and reproducti on in any medium, provided the original work is properly cited  \n International Journal of Scientific Research in Computer Science, Engineering and Information Technology'),
 Document(metadata={'page': 4, 'source': './backend/app/upload_files/Identification_of_Gender_from_Facial_Features.pdf'}, page_content='difference  of the second variable onto the first \nvariable by translation and rotation of original axes \nand projecting data onto new axes. The direction of \nprojection is decided using eige nvalues and \neigenvectors. So, the first few modified  features \n(termed as Pri

In [12]:
docs_to_return = [
            {
                'metadata': doc.metadata,
                'page_content': doc.page_content
            }
            for doc in docs
        ]

In [13]:
docs_to_return

[{'metadata': {'page': 0,
   'source': './backend/app/upload_files/Identification_of_Gender_from_Facial_Features.pdf'},
  'page_content': 'Copyright: ©  the author(s), publisher and licensee Technoscience Academy. This is an open -access article distributed under the \nterms of the Creative Commons Attribution Non -Commercial License, which permits unrestricted non -commercial use, \ndistribution, and reproducti on in any medium, provided the original work is properly cited  \n International Journal of Scientific Research in Computer Science, Engineering and Information Technology'},
 {'metadata': {'page': 4,
   'source': './backend/app/upload_files/Identification_of_Gender_from_Facial_Features.pdf'},
  'page_content': 'difference  of the second variable onto the first \nvariable by translation and rotation of original axes \nand projecting data onto new axes. The direction of \nprojection is decided using eige nvalues and \neigenvectors. So, the first few modified  features \n(termed 

In [18]:
from langchain_community.llms import Ollama
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

llm = Ollama(model="phi")
prompt_template = """Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

prompt = PromptTemplate(template=prompt_template, input_variables=['context', 'question'])

chain_type_kwargs = {"prompt": prompt}

try:
    qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever_ensemble, return_source_documents=True, chain_type_kwargs=chain_type_kwargs, verbose=True)
    
    def get_response(input):
        query = input
        response = qa(query)
        # docs = vector_store.similarity_search_with_score(query)
        result = 'Answer:' + response['result']
        # +'\n' + 'Similarity_score: '+ str(docs[0][1])
        return result

    print(get_response("what is title of the document?"))

except ConnectionRefusedError as e:
    print(f"ConnectionRefusedError: {e}")
except Exception as e:
    print(f"An error occurred: {e}")

c:\Users\Asus\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new RetrievalQA chain...

> Finished chain.
Answer: The title of the document is "ĐẠI HỌC QUốC GIA TP.HCM TRƯỜNG ĐẠI HỌC CÔNG NGHỆ THÔNG TIN CỘNG HòA XÃ HỘI CHỦ NGHĨA VIỆT NAM Độc lập 
-Ban hành kèm theo Quyết định số: 572 
/QĐ-ĐHCNTT ngày 29 tháng 9 năm 2021

"""

print(doc) #print all lines of doc using .splitlines() method



In [1]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

llm = Ollama(
    model="llama3", callback_manager=CallbackManager([StreamingStdOutCallbackHandler()])
)
llm("The first man on the summit of Mount Everest, the highest peak on Earth, was ...")

NameError: name 'Ollama' is not defined